In [1]:
import json, torch
import lightning  as L
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from torch.nn import functional as F
import csv
from collections import defaultdict
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

/home/davidteng/anaconda3/envs/wsd/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f"Using GPU: {torch.cuda.is_available()}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Using GPU: True
cuda:0


#### load training data

In [3]:
train_data_paths = [f"../36_TrainingData/L{loc}_Train.csv" for loc in range(1, 17+1)]
add_ids = [2,4,7,8,9,10,12]
add_train_data_paths = [f"../36_TrainingData_Additional_V2/L{loc}_Train_2.csv" for loc in add_ids]

train_data_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for csv_path in train_data_paths:
    with open(csv_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        header = next(reader)  # 跳過標題列
        for row in reader:
            # 解析數據並存入字典
            loc = row[0]
            date = row[1][:10]
            time = row[1][11:16]
            train_data_dict[loc][date][time] = row[2:]
print(len(train_data_dict['2']))
for csv_path in add_train_data_paths:
    with open(csv_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        header = next(reader)  # 跳過標題列
        for row in reader:
            # 解析數據並存入字典
            loc = row[0]
            date = row[1][:10]
            time = row[1][11:16]
            train_data_dict[loc][date][time] = row[2:]
print((train_data_dict['2']['2024-01-17']['15:28']))

133
['0', '1013.49', '26.46', '42.99', '18575.83', '61.29']


#### load cwb data

In [4]:
months = ['01','02','03','04','05','06','07','08', '09', '10']
features = ['rain', 'raintime', 'solarpower', 'suntime', 'temp', 'uv']
cwb_data_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
for month in months:
    for feature in features:
        with open(f'../cwbdata/{month}/{feature}-{month}.csv', 'r', encoding='utf-8') as csv_file:
            reader = csv.reader(csv_file)
            for row in reader:
                if row[0].isdigit():
                    cwb_data_dict[feature][month][row[0]] = row[1:]
print(cwb_data_dict['rain']['01']['01'])

['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0']


#### preprocess data

In [5]:
class Cwb2LocDataset(Dataset):
    def __init__(self, train_data_dict, cwb_data_dict, loc, max_len=700):
        self.x = []
        self.y = []
        for location, loc_dict in train_data_dict.items():
            # if location != loc:
            #     continue
            for date, date_dict in loc_dict.items():
                train_date_x = []
                train_date_y = []
                month = date[5:7]
                day = date[8:10]
                for time, values in date_dict.items():
                    hour = time[0:2]
                    try:
                        # 確保索引不超出範圍，逐項處理每個特徵值
                        newx = [
                            float(cwb_data_dict['rain'][month][day][int(hour) + 1]) if month in cwb_data_dict['rain'] and day in cwb_data_dict['rain'][month] and int(hour) + 1 < len(cwb_data_dict['rain'][month][day]) else 0.0,
                            float(cwb_data_dict['raintime'][month][day][int(hour) + 1]) if month in cwb_data_dict['raintime'] and day in cwb_data_dict['raintime'][month] and int(hour) + 1 < len(cwb_data_dict['raintime'][month][day]) else 0.0,
                            float(cwb_data_dict['solarpower'][month][day][int(hour) + 1]) if month in cwb_data_dict['solarpower'] and day in cwb_data_dict['solarpower'][month] and int(hour) + 1 < len(cwb_data_dict['solarpower'][month][day]) else 0.0,
                            float(cwb_data_dict['suntime'][month][day][int(hour) + 1]) if month in cwb_data_dict['suntime'] and day in cwb_data_dict['suntime'][month] and int(hour) + 1 < len(cwb_data_dict['suntime'][month][day]) else 0.0,
                            float(cwb_data_dict['temp'][month][day][int(hour) + 1]) if month in cwb_data_dict['temp'] and day in cwb_data_dict['temp'][month] and int(hour) + 1 < len(cwb_data_dict['temp'][month][day]) else 0.0,
                            float(cwb_data_dict['uv'][month][day][int(hour) + 1]) if month in cwb_data_dict['uv'] and day in cwb_data_dict['uv'][month] and int(hour) + 1 < len(cwb_data_dict['uv'][month][day]) else 0.0,
                            int(location)
                        ]
                    except (IndexError, KeyError, ValueError, TypeError):
                        newx = [0.0] * 7  # 這部分只在其他未知錯誤時填充全 0

                    newy = self.str2float(values[-1:])
                    train_date_x.append(newx)
                    train_date_y.append(newy)
                
                # 填充到 max_len
                if len(train_date_x) < max_len:
                    padding_length = max_len - len(train_date_x)
                    train_date_x.extend([[0.0] * 7] * padding_length)
                    train_date_y.extend([[0.0] * 1] * padding_length)
                else:
                    train_date_x = train_date_x[:max_len]
                    train_date_y = train_date_y[:max_len]
                
                self.x.append(train_date_x)
                self.y.append(train_date_y)
        
        # 檢查所有樣本長度是否一致
        for sample_x in self.x:
            assert len(sample_x) == max_len, f"Sample X length {len(sample_x)} != {max_len}"
        for sample_y in self.y:
            assert len(sample_y) == max_len, f"Sample Y length {len(sample_y)} != {max_len}"
        
        # 將數據轉換為 NumPy 數組
        self.x = np.array(self.x, dtype=np.float32)  # 形狀: (num_samples, max_len, 6)
        self.y = np.array(self.y, dtype=np.float32)  # 形狀: (num_samples, max_len, 4)
        
        # 初始化標準化器
        self.x_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        
        # 重塑數據以適應 StandardScaler 的輸入要求 (num_samples * max_len, num_features)
        num_samples, max_len_x, num_features_x = self.x.shape
        num_samples_y, max_len_y, num_features_y = self.y.shape
        assert max_len_x == max_len_y, "x 和 y 的序列長度不一致"
        
        self.x = self.x.reshape(-1, num_features_x)
        self.y = self.y.reshape(-1, num_features_y)
        
        # 擬合並轉換數據
        self.x_scaler.fit(self.x)
        self.y_scaler.fit(self.y)
        
        self.x = self.x_scaler.transform(self.x).reshape(num_samples, max_len_x, num_features_x)
        self.y = self.y_scaler.transform(self.y).reshape(num_samples, max_len_y, num_features_y)
        
        # 將數據轉換回列表以節省內存（可選）
        # self.x = self.x.tolist()
        # self.y = self.y.tolist()

        # 保存標準化器
        with open(f'./scalar/x_scaler.pkl', 'wb') as f:
            pickle.dump(self.x_scaler, f)

        with open(f'./scalar/y_scaler.pkl', 'wb') as f:
            pickle.dump(self.y_scaler, f)
    
    def __getitem__(self, index):
        return torch.tensor(self.x[index], dtype=torch.float32), torch.tensor(self.y[index], dtype=torch.float32)
    
    def __len__(self):
        return len(self.x)
    
    def str2float(self, string_list):
        float_list = []
        for x in string_list:
            try:
                float_list.append(float(x))
            except (ValueError, TypeError):
                float_list.append(0.0)
        return float_list


In [6]:
# test = Cwb2LocDataset(
#     train_data_dict,
#     cwb_data_dict,
#     loc='1',
# )
# print(test[1][0][:10]) # train_x
# print(test[1][1][:10]) # train_y
# print(len(test[0][0]))

#### cut data

In [7]:
class Cwb2LocDataModule(L.LightningDataModule):
    def __init__(self, train_data_dict, cwb_data_dict, loc, batch_size=16, train_split=0.9):
        super().__init__()
        self.train_data_dict = train_data_dict
        self.cwb_data_dict = cwb_data_dict
        self.batch_size = batch_size
        self.train_split = train_split
        self.loc = loc

    def setup(self, stage=None):
        dataset = Cwb2LocDataset(self.train_data_dict, self.cwb_data_dict, str(self.loc))
        train_size = int(len(dataset) * self.train_split)
        val_size = len(dataset) - train_size
        self.train_dataset, self.val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

In [8]:
# test_datamodule = Cwb2LocDataModule(train_data_dict, cwb_data_dict, 1)
# test_datamodule.setup()
# train_loader = test_datamodule.train_dataloader()
# val_loader = test_datamodule.val_dataloader()

# print(f"Train batches: {len(train_loader)}")
# print(f"Validation batches: {len(val_loader)}")

# # 測試加載數據
# for batch in train_loader:
#     x, y = batch
#     print(f"Train batch features shape: {x.shape}")
#     print(f"Train batch labels shape: {y.shape}")
#     break

# for batch in val_loader:
#     x, y = batch
#     print(f"Validation batch features shape: {x.shape}")
#     print(f"Validation batch labels shape: {y.shape}")
#     break

In [9]:
class Cwb2LocModel(L.LightningModule):
    def __init__(self, input_dim=7, hidden_dim=128, num_layers=2, output_dim=1, learning_rate=1e-3, delta=1.0):
        super(Cwb2LocModel, self).__init__()
        self.save_hyperparameters()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.delta = delta  # 動態設置 delta

        # 定義 LSTM 層
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2, bidirectional=True)

        # 定義全連接層
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim // 4)
        self.fc2 = nn.Linear(hidden_dim // 4, input_dim)
        self.fc3 = nn.Linear(input_dim, 1)

        # 定義損失函數（Huber Loss）
        # self.criterion = nn.HuberLoss(delta=self.delta)
        # self.criterion = nn.MSELoss()
        self.criterion = nn.L1Loss()
        # self.criterion = log_cosh_loss

    def log_cosh_loss(y_pred, y_true):
        loss = torch.mean(torch.log(torch.cosh(y_pred - y_true)))
        return loss

    def forward(self, x):
        # 初始化隱藏狀態和細胞狀態
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(self.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(self.device)
        residual = x
        
        # 前向傳播 LSTM
        out, _ = self.lstm(x, (h0, c0))

        # 通過全連接層得到最終輸出
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out += residual
        out = self.fc3(out)
        
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)  # 使用 Huber Loss
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)  # 使用 Huber Loss
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(
            self.parameters(),
            lr=self.hparams.learning_rate,
            weight_decay=1e-2
        )
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.9)
        return [optimizer], [scheduler]


In [10]:
batch_size = 16
learning_rate=1e-3
# loc = 2
hidden_dim = 128
lstm_layers = 2
data_module = Cwb2LocDataModule(train_data_dict, cwb_data_dict, loc, batch_size)
model = Cwb2LocModel(
    input_dim=7,
    hidden_dim=hidden_dim,
    num_layers=lstm_layers,
    output_dim=1,
    learning_rate=learning_rate
)
wandb_name = f'v4_all_b{batch_size}_h{hidden_dim}_ll{lstm_layers}_lr{learning_rate}_雙向_mae_p16_3fc_test00'
# dirpath=f'./saved_models/{loc}/{wandb_name}'
# dirpath=f'./saved_models/{loc}'
dirpath=f'./saved_models'

In [11]:
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping
logger = WandbLogger(
    project='aicup_power_v2', 
    name=wandb_name,
    save_dir=None,
    offline=False,
    log_model=False,  # 不保存模型
    save_code=False   # 不保存代碼快照
)
callbacks = [
    LearningRateMonitor(),
    ModelCheckpoint(
        monitor='val_loss',
        mode='min',
        auto_insert_metric_name=False,
        dirpath=dirpath,
        filename='best-checkpoint-res-2-128-mae',
        save_top_k=1,
    ),
    EarlyStopping(
        monitor='val_loss',   # 監控的指標
        patience=50,           # 如果 3 個 epoch 驗證損失沒有改善，則停止訓練
        verbose=True,
        mode='min'            # 損失越小越好
    )
]

In [12]:
trainer = L.Trainer(
    precision=32,
    gradient_clip_val=1.0,
    accumulate_grad_batches=1,
    max_epochs=10000,
    val_check_interval=1.0,
    logger=logger,
    callbacks=callbacks,
)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [13]:
trainer.fit(model, datamodule=data_module)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: david___teng (david___teng-national-central-university). Use `wandb login --relogin` to force relogin


/home/davidteng/anaconda3/envs/wsd/lib/python3.9/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/davidteng/aicup/electric/code/saved_models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type   | Params | Mode 
---------------------------------------------
0 | lstm      | LSTM   | 535 K  | train
1 | fc1       | Linear | 8.2 K  | train
2 | fc2       | Linear | 231    | train
3 | fc3       | Linear | 8      | train
4 | criterion | L1Loss | 0      | train
---------------------------------------------
544 K     Trainable params
0         Non-trainable params
544 K     Total params
2.176     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/davidteng/anaconda3/envs/wsd/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.


/home/davidteng/anaconda3/envs/wsd/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.


...

Epoch 176: 100%|██████████| 121/121 [00:02<00:00, 47.89it/s, v_num=oeze, train_loss_step=0.103, val_loss=0.176, train_loss_epoch=0.121] 

Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.176


Epoch 226: 100%|██████████| 121/121 [00:02<00:00, 48.14it/s, v_num=oeze, train_loss_step=0.129, val_loss=0.177, train_loss_epoch=0.106] 

Monitored metric val_loss did not improve in the last 50 records. Best score: 0.176. Signaling Trainer to stop.


Epoch 226: 100%|██████████| 121/121 [00:02<00:00, 48.06it/s, v_num=oeze, train_loss_step=0.129, val_loss=0.177, train_loss_epoch=0.106]
